Billboard Hot 100 Chart Overview

In [2]:
# Import all libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import time
from datetime import datetime, timedelta
import plotly.express as px

In [3]:
# Generate Saturdays from a specific year until today's date
def generate_saturdays(start_year):
    saturdays = []
    start_date = datetime(start_year, 1, 1)
    end_date = datetime.now()

    # Loop through dates until today
    while start_date <= end_date:
        if start_date.weekday() == 5:
            saturdays.append(start_date.strftime('%Y-%m-%d'))
        start_date += timedelta(days=1)

    return saturdays

# Generate Saturdays starting from 2022 until today
saturdays = generate_saturdays(2022)
print(saturdays)

['2022-01-01', '2022-01-08', '2022-01-15', '2022-01-22', '2022-01-29', '2022-02-05', '2022-02-12', '2022-02-19', '2022-02-26', '2022-03-05', '2022-03-12', '2022-03-19', '2022-03-26', '2022-04-02', '2022-04-09', '2022-04-16', '2022-04-23', '2022-04-30', '2022-05-07', '2022-05-14', '2022-05-21', '2022-05-28', '2022-06-04', '2022-06-11', '2022-06-18', '2022-06-25', '2022-07-02', '2022-07-09', '2022-07-16', '2022-07-23', '2022-07-30', '2022-08-06', '2022-08-13', '2022-08-20', '2022-08-27', '2022-09-03', '2022-09-10', '2022-09-17', '2022-09-24', '2022-10-01', '2022-10-08', '2022-10-15', '2022-10-22', '2022-10-29', '2022-11-05', '2022-11-12', '2022-11-19', '2022-11-26', '2022-12-03', '2022-12-10', '2022-12-17', '2022-12-24', '2022-12-31', '2023-01-07', '2023-01-14', '2023-01-21', '2023-01-28', '2023-02-04', '2023-02-11', '2023-02-18', '2023-02-25', '2023-03-04', '2023-03-11', '2023-03-18', '2023-03-25', '2023-04-01', '2023-04-08', '2023-04-15', '2023-04-22', '2023-04-29', '2023-05-06', '2023

In [4]:
driver = webdriver.Firefox()

year = []
month = []
day = []

title = []
singer = []
last_week_position = []
peak_position = []
weeks_on_chart = []
current_week_position = []


for date_str in saturdays:
    url = f'https://www.billboard.com/charts/hot-100/{date_str}/'
    driver.get(url)
    
    w = WebDriverWait(driver, 15)
    w.until(EC.presence_of_element_located((By.CLASS_NAME,"c-title")))
    
    song_titles = driver.find_elements(By.ID, "title-of-a-story") ## Find the common class across the list of music
    
    i=0
    
    for song_title in song_titles:
        # Dates
        date = datetime.strptime(date_str, '%Y-%m-%d')  # Convert date string back to datetime
        month.append(date.strftime("%m"))  # Append the month
        day.append(date.strftime("%d"))     # Append the day
        year.append(date.strftime("%Y"))    # Append the year
        
        i = i+1

        try:
            title_element = driver.find_element(By.XPATH, f'/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[{i}]/ul/li[4]/ul/li[1]/h3').text
            title.append(title_element)
        except:
            title.append("No title available")
            pass
        
        # Extract Singer
        try:
            singer_element = driver.find_element(By.XPATH, f'//*[@id="post-1479786"]/div[3]/div/div/div/div[2]/div[{i}]/ul/li[4]/ul/li[1]/span').text
            singer.append(singer_element)
        except:
            singer.append("No singer available")
            pass
            

        # This weak position
        try:
            current_week_pos = driver.find_element(By.XPATH, f'//*[@id="post-1479786"]/div[3]/div/div/div/div[2]/div[{i}]/ul/li[1]/span').text
            current_week_position.append(current_week_pos)
        except:
            current_week_position.append("No Week Position")
            pass


        # Extract last week
        try:
            last_week_pos = driver.find_element(By.XPATH, f'/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[{i}]/ul/li[4]/ul/li[4]/span').text
            last_week_position.append(last_week_pos)
        except:
            last_week_position.append("No Last Week Position")
            pass

        # Peak Position
        try:
            peak_pos = driver.find_element(By.XPATH, f'/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[{i}]/ul/li[4]/ul/li[5]/span').text
            peak_position.append(peak_pos)
        except:
            peak_position.append("No Peak Position")
            pass

        # Weeks on Chart
        try:
            wks_chart = driver.find_element(By.XPATH, f'//*[@id="post-1479786"]/div[3]/div/div/div/div[2]/div[{i}]/ul/li[4]/ul/li[6]/span').text
            weeks_on_chart.append(wks_chart)
        except:
            weeks_on_chart.append("No Weeks on Chart")
            pass
            

driver.quit() 

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [5]:
df = pd.DataFrame(year, columns=['Year'])
df['Month'] = month
df['Day'] = day
df['Rank'] = current_week_position
df['Title'] = title
df['Singer(s)'] = singer
df['Last Week Position'] = last_week_position
df['Peak Position'] = peak_position
df['Weeks on Chart'] = weeks_on_chart

df = df.loc[df['Title'] != 'No title available'].reset_index(drop=True)

df


,Year,Month,Day,Rank,Title,Singer(s),Last Week Position,Peak Position,Weeks on Chart
0,2022,01,01,1,All I Want For Christmas Is You,Mariah Carey,1,1,50
1,2022,01,01,2,Rockin' Around The Christmas Tree,Brenda Lee,2,2,44
2,2022,01,01,3,Jingle Bell Rock,Bobby Helms,4,3,41
3,2022,01,01,4,A Holly Jolly Christmas,Burl Ives,5,4,25
4,2022,01,01,5,Easy On Me,Adele,3,1,11
...,...,...,...,...,...,...,...,...,...
14268,2024,10,12,96,Keep Up,Odetari,-,96,1
14269,2024,10,12,97,Passport Junkie,Rod Wave,98,61,3
14270,2024,10,12,98,This Is My Dirt,Justin Moore,-,98,1
14271,2024,10,12,99,Close To You,Gracie Abrams,-,49,9


In [6]:
# Convert columns to numeric
df['Rank'] = pd.to_numeric(df['Rank'])
df['Last Week Position'] = pd.to_numeric(df['Last Week Position'], errors='coerce')
df['Peak Position'] = pd.to_numeric(df['Peak Position'], errors='coerce')
df['Weeks on Chart'] = pd.to_numeric(df['Weeks on Chart'], errors='coerce')

# Display the DataFrame
df

,Year,Month,Day,Rank,Title,Singer(s),Last Week Position,Peak Position,Weeks on Chart
0,2022,01,01,1,All I Want For Christmas Is You,Mariah Carey,1.0,1.0,50
1,2022,01,01,2,Rockin' Around The Christmas Tree,Brenda Lee,2.0,2.0,44
2,2022,01,01,3,Jingle Bell Rock,Bobby Helms,4.0,3.0,41
3,2022,01,01,4,A Holly Jolly Christmas,Burl Ives,5.0,4.0,25
4,2022,01,01,5,Easy On Me,Adele,3.0,1.0,11
...,...,...,...,...,...,...,...,...,...
14268,2024,10,12,96,Keep Up,Odetari,NaN,96.0,1
14269,2024,10,12,97,Passport Junkie,Rod Wave,98.0,61.0,3
14270,2024,10,12,98,This Is My Dirt,Justin Moore,NaN,98.0,1
14271,2024,10,12,99,Close To You,Gracie Abrams,NaN,49.0,9


Top 10 Songs by Accumulated Weeks at No. 1 (2022-2024)

In [7]:
# Filter for songs that were No. 1
no1_songs = df[df['Rank'] == 1]

# Create a new DataFrame with just the Year, Title, and Singer(s)
# Group by Title and Singer(s) to count the weeks at No. 1
accumulated_weeks_no1 = no1_songs.groupby(['Year', 'Title', 'Singer(s)']).size().reset_index(name='Accumulated Weeks')
# Sort by accumulated weeks at No. 1
accumulated_weeks_no1 = accumulated_weeks_no1.sort_values(by=['Accumulated Weeks'], ascending=False)
# Show the top 10 songs with the most accumulated weeks at No.1
top_10_no1 = accumulated_weeks_no1.head(10)

# Display the final DataFrame
print(top_10_no1)

    Year                            Title  \
22  2023                       Last Night   
3   2022                        As It Was   
32  2024               A Bar Song (Tipsy)   
18  2023                          Flowers   
36  2024                  I Had Some Help   
2   2022                        Anti-Hero   
8   2022                       Heat Waves   
12  2022        We Don't Talk About Bruno   
1   2022  All I Want For Christmas Is You   
39  2024                      Lovin On Me   

                                            Singer(s)  Accumulated Weeks  
22                                      Morgan Wallen                 16  
3                                        Harry Styles                 15  
32                                          Shaboozey                 13  
18                                        Miley Cyrus                  8  
36                Post Malone Featuring Morgan Wallen                  6  
2                                        Taylor Swift 

In [33]:
# Top 10 Songs by Accumulated Weeks at No. 1 (2022-2024)

# Create a bar chart
fig = px.bar(
    top_10_no1,
    x='Title',
    y='Accumulated Weeks',
    hover_data=['Singer(s)'],
    color='Accumulated Weeks',
    title='Top 10 Songs by Most Accumulated Weeks at No. 1 (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    xaxis_title='Songs',
    yaxis_title='Accumulated Weeks at No.1',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        tickfont=dict(size=16),
        tickmode='linear',
        dtick=2
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()


Insights

The analysis of the top 10 songs by accumulated weeks at No. 1 from 2022 to 2024 highlights key trends in the music industry. Morgan Wallen’s "Last Night" leads with 16 weeks at No. 1, demonstrating his significant presence in the charts, while Harry Styles’ "As It Was," with 15 weeks, shows his strong appeal in contemporary pop. Shaboozey's "A Bar Song (Tipsy)" reaching 13 weeks indicates a growing space for new artists to make an impact. The success of Miley Cyrus with "Flowers" and Taylor Swift’s "Anti-Hero" reflects a notable presence of female artists in top positions, suggesting a shift toward greater representation.

Collaborations are also noteworthy, as seen with Post Malone and Morgan Wallen’s "I Had Some Help," illustrating how partnerships can enhance chart performance. Additionally, Mariah Carey’s "All I Want For Christmas Is You" remains a standout classic, achieving five weeks at No. 1 during its seasonal peaks. Its enduring popularity highlights the significant impact of holiday music on the Billboard Hot 100, showcasing how certain songs can maintain relevance and success year after year. Overall, this data reveals a mix of established artists and emerging talent, pointing to a changing landscape in popular music.

This data reveals a mix of established artists and emerging talent, indicating a shifting landscape in popular music. The blend of established and emerging talent points to a dynamic environment where popular music is evolving. As new artists gain traction, they challenge the dominance of established acts, leading to a more varied and inclusive music scene. This shift may result in broader genre representation and innovative collaborations, ultimately influencing future trends in the industry.

Monthly New Songs in the Top 10 (2022-2024)

In [34]:
# Filter for new songs that were in Top 10
top10_songs = df[(df['Rank'] <= 10) & (df['Last Week Position'] > 10)]

# Create a new DataFrame with just the Year, Month, Title, and Singer(s)
# Group by Year and Month to count the number of Top 10 songs
top10_songs_by_month = top10_songs.groupby(['Year', 'Month']).size().reset_index(name='Monthly New Songs in the Top 10')
# Combine Year and Month
top10_songs_by_month['Year-Month'] = top10_songs_by_month['Year'] + '-' + top10_songs_by_month['Month']
# Drop the original Year and Month columns
top10_songs_by_month = top10_songs_by_month.drop(columns=['Year', 'Month'])
# Rearrange the columns so that 'Year and Month' comes first
top10_songs_by_month = top10_songs_by_month[['Year-Month', 'Monthly New Songs in the Top 10']]

# Display the final DataFrame
print(top10_songs_by_month)

   Year-Month  Monthly New Songs in the Top 10
0     2022-01                               12
1     2022-02                                4
2     2022-03                                4
3     2022-04                                2
4     2022-05                                2
5     2022-06                                4
6     2022-07                                4
7     2022-08                                3
8     2022-09                                4
9     2022-10                                3
10    2022-11                                9
11    2022-12                                7
12    2023-01                               10
13    2023-02                                5
14    2023-03                                5
15    2023-04                                4
16    2023-05                                4
17    2023-06                                1
18    2023-07                                2
19    2023-08                                4
20    2023-09

,Year-Month,Monthly New Songs in the Top 10
0,2022-01,12
1,2022-02,4
2,2022-03,4
3,2022-04,2
4,2022-05,2
5,2022-06,4
6,2022-07,4
7,2022-08,3
8,2022-09,4
9,2022-10,3


In [45]:
# Monthly New Songs in the Top 10 (2022-2024)

# Create a line chart
fig = px.line(
    top10_songs_by_month,
    x='Year-Month',
    y='Monthly New Songs in the Top 10',
    color_discrete_sequence=['red'],
    title='Monthly New Songs in the Top 10 (2022-2024)',
)

# Customize the chart
fig.update_traces(hovertemplate='Year-Month: %{x|%Y-%m}<br>Value: %{y}')
fig.update_layout(
    xaxis_title='Month-Year',
    yaxis_title='Counts',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ))

# Show the chart
fig.show()


Insights

The data for monthly new songs in the Top 10 from 2022 to 2024 reveals intriguing trends in music releases, particularly the surge at the beginning of the year. January 2022 saw a high of 12 new songs entering the Top 10, likely influenced by the popularity of holiday music from December, as listeners continue to engage with festive tracks in the weeks following Christmas. However, this initial influx was followed by a sharp decline in new entries in February and March, indicating a typical lull in new releases after the holiday season. Notably, November and December 2022 also experienced peaks in new songs, suggesting that the holiday season prompts a renewed interest in music releases. Throughout 2023 and into 2024, the data indicates fluctuations, with some months showing very few new entries, particularly during the summer. 

Overall, these trends suggest that strategic timing around holidays and seasonal variations play a significant role in shaping the landscape of new music in the Top 10.

Top 5 Singers with the most numbers of Top 10 Songs (2022-2024)

In [11]:
# Filter for songs that were in the top 10
top10_songs = df[df['Rank'] <= 10]

# Drop duplicates based on 'Title' and 'Singer(s)' to ensure each song is counted once per artist
unique_top10_songs = top10_songs.drop_duplicates(subset=['Title', 'Singer(s)'])

# Group by Singer(s) to count the number of unique top 10 songs
top10_songs_singers = unique_top10_songs.groupby('Singer(s)').size().reset_index(name='No. of Top 10 Songs')

# Sort by the number of top 10 songs
top10_songs_singers = top10_songs_singers.sort_values(by='No. of Top 10 Songs', ascending=False)

# Show the top 5 singers with the most number of unique top 10 songs
top5_singers = top10_songs_singers.head(5)

# Display the final DataFrame
print(top5_singers)

             Singer(s)  No. of Top 10 Songs
108       Taylor Swift                   25
24               Drake                    8
87       Morgan Wallen                    7
25   Drake & 21 Savage                    6
23            Doja Cat                    5


In [12]:
# Top 5 Singers with the most numbers of Top 10 Songs (2022-2024)

# Create a bar chart
fig = px.bar(
    top5_singers,
    x='Singer(s)',
    y='No. of Top 10 Songs',
    color='No. of Top 10 Songs',
    title='Top 5 Singers with The Most Numbers of Top 10 Songs (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    xaxis_title='Singer(s)',
    yaxis_title='No. of Top 10 Songs',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()


Insights

The data on the top singers with the most Top 10 songs from 2022 to 2024 highlights the dominance of Taylor Swift, who leads the list with an impressive 25 entries. This substantial number underscores her ability to consistently produce hits that resonate with audiences, solidifying her position as a major force in the music industry. Following her is Drake, with 8 Top 10 songs, demonstrating his enduring popularity and versatility as an artist. Morgan Wallen, with 7 entries, reflects the growing influence of country music in the mainstream. The collaboration between Drake and 21 Savage resulted in 6 Top 10 songs, emphasizing the effectiveness of partnerships in the current music landscape. Finally, Doja Cat rounds out the list with 5 Top 10 songs, showcasing her unique style and appeal. 

Overall, this data illustrates a competitive environment where established artists continue to thrive while adapting to changing musical trends.

No.1 Songs with Most Weeks on Chart (2022-2024)

In [13]:
# Filter for songs that reached No. 1
no1_songs = df[df['Rank'] == 1]

# Group the songs by Title and Singer(s) and count the maximum weeks
weeks_count = no1_songs.groupby(['Title', 'Singer(s)'])['Weeks on Chart'].max().reset_index()

# Count how many songs spend each number of weeks on chart
weeks_to_no1_count = weeks_count['Weeks on Chart'].value_counts().reset_index()

# Rename the columns for clarity
weeks_to_no1_count.columns = ['Number of Weeks', 'Count of Songs']

# Sort by the number of weeks in ascending order
weeks_to_no1_count_sorted = weeks_to_no1_count.sort_values(by=['Number of Weeks'], ascending=False)

# Show the top 10 results
top10_weeks_to_no1_count_sorted = weeks_to_no1_count_sorted.head(10)

# Display the final DataFrame
print(top10_weeks_to_no1_count_sorted)


    Number of Weeks  Count of Songs
18               64               1
12               63               1
19               58               1
16               32               1
10               31               1
8                28               2
4                25               2
14               19               1
3                15               3
17               14               1


In [14]:
# No.1 Songs with Most Weeks on Chart (2022-2024)

# Create a bar chart
fig = px.bar(
    top10_weeks_to_no1_count_sorted,
    x='Number of Weeks',
    y='Count of Songs',
    color_discrete_sequence=['#E48F72'],
    title='No.1 Songs with Most Weeks on Chart (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=24),
    xaxis_title='Number of Weeks',
    yaxis_title='Count of Songs',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16),
        type='category'
    ),
    yaxis=dict(
        tickfont=dict(size=16),
        tickmode='linear'
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()


Insights

The data on No. 1 songs with the most weeks on the chart from 2022 to 2024 reveals significant patterns in song longevity and popularity. Notably, there is one song that spent an impressive 64 weeks on the chart, indicating a remarkable level of sustained success. Another standout track followed closely behind with 63 weeks, further emphasizing the potential for certain songs to capture and maintain audience interest over an extended period.

The distribution of weeks shows a clear decline in the number of songs as the weeks increase, with only a few tracks managing to remain in the spotlight for 32 weeks or more. For instance, one song achieved 58 weeks, while another spent 31 weeks at No. 1, reflecting the competitive nature of the charts where few can achieve such longevity.

Interestingly, there are two songs that spent 28 weeks in the top position for 8 weeks, indicating a solid, albeit less dominant, performance. As the weeks decrease, the count of songs increases, suggesting that while fewer songs manage to maintain a long presence, many can achieve shorter bursts of popularity. 

Overall, this data highlights the dynamics of the music charts, where certain tracks can achieve remarkable staying power, while others capture attention for shorter durations.

Songs with the most weeks on chart (2022-2024)

In [15]:
# Group the songs by Title and Singer(s) and count the maximum weeks
weeks_on_chart = df.groupby(['Title', 'Singer(s)'])['Weeks on Chart'].max().reset_index()

# Sort by the number of weeks in ascending order
weeks_on_chart_sorted = weeks_on_chart.sort_values(by=['Weeks on Chart'], ascending=False)

# Show the top 10 results
top10_weeks_on_chart_sorted = weeks_on_chart_sorted.head(10)

# Display the final DataFrame
print(top10_weeks_on_chart_sorted)


                                Title                      Singer(s)  \
678                        Heat Waves                  Glass Animals   
905                        Levitating                       Dua Lipa   
1440                           Snooze                            SZA   
1359                  Save Your Tears     The Weeknd & Ariana Grande   
1449          Something In The Orange                     Zach Bryan   
62    All I Want For Christmas Is You                   Mariah Carey   
1482                             Stay  The Kid LAROI & Justin Bieber   
1831                        You Proof                  Morgan Wallen   
105                         As It Was                   Harry Styles   
939                      Lose Control                    Teddy Swims   

      Weeks on Chart  
678               91  
905               77  
1440              70  
1359              69  
1449              66  
62                65  
1482              63  
1831              62  


In [16]:
# Songs with the most weeks on chart (2022-2024)
# Create a bar chart
fig = px.bar(
    top10_weeks_on_chart_sorted,
    x='Title',
    y='Weeks on Chart',
    color='Weeks on Chart',
    hover_data=['Singer(s)'],
    title='Songs with The Most Weeks on Chart (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=24),
    title_x=0.5,
    xaxis_title='Songs',
    yaxis_title='Weeks on Chart',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16),
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()

The data on songs with the most weeks on the chart from 2022 to 2024 highlights notable trends in song longevity and audience engagement. "Heat Waves" by Glass Animals leads the list with an impressive 91 weeks on the chart, showcasing its enduring popularity and broad appeal. Following closely is Dua Lipa’s "Levitating," which spent 77 weeks on the chart, reinforcing her status as a key figure in contemporary pop music.

SZA's "Snooze" and The Weeknd's collaboration with Ariana Grande on "Save Your Tears" both maintained strong performances with 70 and 69 weeks, respectively. These tracks reflect the growing influence of R&B and pop collaborations in sustaining listener interest over time. Similarly, Zach Bryan’s "Something In The Orange," with 66 weeks, highlights the increasing crossover appeal of country music in the mainstream.

Mariah Carey’s "All I Want For Christmas Is You" continues to demonstrate its timelessness with 65 weeks, emphasizing the unique position of holiday music in chart longevity. Additionally, "Stay" by The Kid LAROI and Justin Bieber, with 63 weeks, illustrates the effectiveness of collaborations in reaching wider audiences. 

Overall, this data reveals a competitive landscape where a mix of established hits and newer tracks can maintain significant chart presence, indicating varied listener preferences and the dynamic nature of popular music over the past two years.

Distribution of Last Week Positions among Top 10 Songs (2022-2024)

In [17]:
# Filter for songs that reached top 10 and were outside that the week before
top10_songs = df[(df['Rank'] <= 10) & (df['Last Week Position'] > 10)]

# Count how many songs took each number of weeks to get to No. 1
last_week_position = top10_songs['Last Week Position'].value_counts().reset_index()

# Rename the columns for clarity
last_week_position.columns = ['Last Week Position', 'Count of Songs']

# Sort by the number of weeks in ascending order
last_week_position_sorted = last_week_position.sort_values(by=['Last Week Position'], ascending=True)

# Show the top 10 results
last_week_position_sorted = last_week_position_sorted.head(20)

# Display the final DataFrame
print(last_week_position_sorted)

    Last Week Position  Count of Songs
0                 11.0              41
1                 12.0              25
3                 13.0              10
4                 14.0               9
2                 15.0              12
7                 16.0               5
6                 17.0               6
10                18.0               3
5                 19.0               7
12                20.0               3
18                21.0               2
8                 22.0               4
16                23.0               2
24                24.0               1
13                25.0               2
17                26.0               2
9                 27.0               4
23                28.0               1
25                29.0               1
19                30.0               1


In [18]:
# Distribution of Last Week Positions among Top 10 Songs (2022-2024)
# Create a histogram
fig = px.histogram(
    last_week_position_sorted,
    x='Last Week Position',
    y='Count of Songs',
    color='Last Week Position',
    title='Distribution of Last Week Positions among Top 10 Songs (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=24),
    title_x=0.5,
    xaxis_title='Last Week Position',
    yaxis_title='Count of Songs',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16),
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()

Insights

The data on the distribution of last week positions among Top 10 songs from 2022 to 2024 reveals insights into the dynamics of song rankings and their stability on the charts. The most common last week position is 11th, with 41 songs recorded, indicating that many popular tracks tend to fall just outside the Top 10 after their peak performance. 

Following this, 25 songs were found in the 12th position, suggesting that while many songs achieve high rankings, a significant number may struggle to maintain their positions in the competitive Top 10. The frequency of songs in the 13th and 15th positions, with 10 and 12 songs, respectively, further highlights the volatility of chart placements, as tracks often hover around the Top 10 but may not sustain their momentum.

Additionally, there are fewer songs in lower positions, such as 24th and 30th, indicating that while many songs can reach the Top 10, fewer maintain consistent presence further down the chart. This distribution illustrates the challenges artists face in staying relevant and competitive in a rapidly changing musical landscape. 

Overall, the data points to a competitive environment where songs may have short bursts of success, often dropping out of the Top 10 quickly, reflecting the dynamic nature of listener preferences and chart performance in recent years.

Peak Position vs. Total Weeks on Chart (2022-2024)

In [42]:
# Group the songs by Title, Singer(s), and Peak Position, and get the maximum weeks
weeks_on_chart = df.groupby(['Title', 'Singer(s)', 'Peak Position'])['Weeks on Chart'].max().reset_index()

# Sort the DataFrame by 'Weeks on Chart' in descending order
weeks_on_chart_sorted = weeks_on_chart.sort_values(by='Weeks on Chart', ascending=False)

# Display the final DataFrame
print(weeks_on_chart_sorted)

                        Title                      Singer(s)  Peak Position  \
1379               Heat Waves                  Glass Animals            1.0   
1798               Levitating                       Dua Lipa            2.0   
2744                   Snooze                            SZA            2.0   
2604          Save Your Tears     The Weeknd & Ariana Grande            1.0   
2774  Something In The Orange                     Zach Bryan           10.0   
...                       ...                            ...            ...   
572             Ch y La Pizza  Fuerza Regida X Natanael Cano           97.0   
1087               First Love    Oscar Ortiz X Edgardo Nunez           94.0   
567                  Ceilings                 Lizzy McAlpine           75.0   
1090               Fish Scale     YoungBoy Never Broke Again           82.0   
1385                   Heaven                    Niall Horan           63.0   

      Weeks on Chart  
1379              91  
1798 

,Title,Singer(s),Peak Position,Weeks on Chart
1379,Heat Waves,Glass Animals,1.0,91
1798,Levitating,Dua Lipa,2.0,77
2744,Snooze,SZA,2.0,70
2604,Save Your Tears,The Weeknd & Ariana Grande,1.0,69
2774,Something In The Orange,Zach Bryan,10.0,66
145,All I Want For Christmas Is You,Mariah Carey,1.0,65
2836,Stay,The Kid LAROI & Justin Bieber,1.0,63
3575,You Proof,Morgan Wallen,5.0,62
205,As It Was,Harry Styles,1.0,61
1855,Lose Control,Teddy Swims,1.0,60


In [38]:
# Peak Position vs. Total Weeks on Chart (2022-2024)
# Create a scatter plot
fig = px.scatter(
    weeks_on_chart_sorted,
    x='Weeks on Chart',
    y='Peak Position',
    color_discrete_sequence=['skyblue'],
    hover_data=['Title', 'Singer(s)'],
    title='Peak Position vs. Total Weeks on Chart (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=24),
    title_x=0.5,
    xaxis_title='Weeks on Chart',
    yaxis_title='Peak Position',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16),
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()

Insights

The data on peak positions and total weeks on the chart from 2022 to 2024 reveals several trends regarding song popularity and longevity. 

"Heat Waves" by Glass Animals stands out with an impressive 91 weeks on the chart and a peak position of No. 1, demonstrating its enduring appeal and consistent listener engagement. Dua Lipa's "Levitating" and SZA's "Snooze," both peaking at No. 2, also show significant staying power with 77 and 70 weeks, respectively, highlighting that songs can achieve long chart runs without necessarily reaching the top position.

The presence of holiday classics like Mariah Carey’s "All I Want For Christmas Is You" and Brenda Lee's "Rockin' Around The Christmas Tree" indicates the lasting impact of seasonal music, both managing to secure long chart durations of 65 and 58 weeks, respectively. 

Morgan Wallen emerges as a dominant artist in this dataset, with multiple songs appearing, such as "You Proof" (62 weeks) and "Last Night" (60 weeks). This reflects his strong presence in contemporary music, appealing to a wide audience.

Overall, this data illustrates a diverse landscape of popular music, where a combination of established hits, seasonal favorites, and emerging artists contribute to chart dynamics. The variety in peak positions and weeks on the chart emphasizes that sustained success can be achieved through both immediate impact and lasting popularity.

Longest Running Songs in Top 10 (2022-2024)

In [21]:
# Filter for songs that reached Top 10
top10_songs = df[df['Rank'] <= 10]

# Count how many weeks each song spent in the Top 10, including the singer
weeks_top10_count = top10_songs.groupby(['Title', 'Singer(s)']).size().reset_index(name='Weeks in Top 10')

# Sort by the number of weeks in descending order
weeks_top10_count_sorted = weeks_top10_count.sort_values(by=['Weeks in Top 10'], ascending=False)

# Show the top 5 results
top_weeks_top10 = weeks_top10_count_sorted.head(5)

# Display the final DataFrame
print(top_weeks_top10)

            Title      Singer(s)  Weeks in Top 10
95     Last Night  Morgan Wallen               41
103  Lose Control    Teddy Swims               37
11      As It Was   Harry Styles               37
35   Cruel Summer   Taylor Swift               34
72     Heat Waves  Glass Animals               30


In [22]:
# Longest Running Songs in Top 10 (2022-2024)
# Create a bar chart
fig = px.bar(
    top_weeks_top10,
    x='Title',
    y='Weeks in Top 10',
    color='Weeks in Top 10',
    hover_data=['Singer(s)'],
    title='Longest Running Songs in Top 10 (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=24),
    title_x=0.5,
    xaxis_title='Songs',
    yaxis_title='Weeks in Top 10',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16),
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart

fig.show()

Insights

The data on the longest-running songs in the Top 10 from 2022 to 2024 highlights several significant trends in music popularity and audience engagement. "Last Night" by Morgan Wallen leads the list with an impressive 41 weeks in the Top 10, underscoring its strong resonance with listeners and its ability to maintain chart presence over time. 

Following closely are Teddy Swims' "Lose Control" and Harry Styles' "As It Was," both with 37 weeks. Their sustained popularity reflects the appeal of contemporary pop and R&B, indicating a broad listener base that supports these tracks over extended periods.

Taylor Swift's "Cruel Summer" also showcases remarkable longevity with 34 weeks in the Top 10, reinforcing her status as a dominant artist in the current music landscape. Lastly, "Heat Waves" by Glass Animals rounds out the list with 30 weeks, further emphasizing its lasting impact on listeners.

Overall, this data illustrates that certain songs can achieve remarkable staying power in the Top 10, indicating not only immediate success but also enduring popularity that captivates audiences over time. This suggests that strong songwriting, memorable performances, and effective marketing can contribute significantly to a song's longevity on the charts.

Top 5 No. 1 Songs with the Fewest Weeks to Reach the Top Position (2022-2024)

In [23]:
# Filter for songs that reached No. 1
no1_songs = df[df['Rank'] == 1]

# Find the weeks on chart for No. 1 songs grouped by title and singer
weeks_to_best = no1_songs.groupby(['Title', 'Singer(s)'])['Weeks on Chart'].min()

# Count how many songs took each number of weeks to get to No. 1
weeks_to_no1_count = weeks_to_best.value_counts().reset_index()

# Rename columns for clarity
weeks_to_no1_count.columns = ['Weeks to No. 1', 'Counts']

# Sort by the number of weeks in ascending order
weeks_to_no1_count_sorted = weeks_to_no1_count.sort_values(by='Weeks to No. 1', ascending=True)

# Get the top 5 results
top5_weeks_to_no1_count = weeks_to_no1_count_sorted.head(5)

# Display the final DataFrame
print(top5_weeks_to_no1_count)


    Weeks to No. 1  Counts
0                1      21
1                2       4
2                4       3
4                5       2
13               6       1


In [24]:
# Top 5 No. 1 Songs with the Fewest Weeks to Reach the Top Position (2022-2024)
# Create a bar chart
fig = px.bar(
    top5_weeks_to_no1_count,
    x='Weeks to No. 1',
    y='Counts',
    color_discrete_sequence=['skyblue'],
    title='Top 5 No. 1 Songs with the Fewest Weeks to Reach the Top Position (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=22),
    xaxis_title='Weeks to No. 1',
    yaxis_title='Counts',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        tickfont=dict(size=12)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()

Insights

The data on the top No. 1 songs with the fewest weeks to reach the top position from 2022 to 2024 reveals intriguing insights into the dynamics of chart performance. A notable 21 songs achieved the No. 1 spot within just one week, indicating that many tracks can make an immediate impact upon release. This suggests that strong marketing strategies, timely releases, or existing fan bases can significantly contribute to a song's rapid ascent.

Additionally, four songs reached No. 1 within two weeks, further demonstrating that a combination of listener interest and effective promotion can lead to quick success. The presence of three songs that took four weeks to hit the top position indicates that while some tracks achieve rapid success, others may require more time to build momentum and audience engagement.

The data also highlights a few outliers, with only two songs taking five weeks and just one song reaching No. 1 after six weeks. This shows that while most hits can climb quickly, a smaller subset may struggle to gain traction initially, reflecting the competitive nature of the music industry.

Overall, this data illustrates the varied paths songs can take to reach the top of the charts, emphasizing the importance of timing, marketing, and listener engagement in achieving chart success.

Monthly New Songs in Hot 100 (2022-2024)

In [44]:
# Filter for new songs
new_songs = df[df['Last Week Position'].isna()]

# Count how many new songs are in each month and year
new_songs_by_month = new_songs.groupby(['Year', 'Month']).size().reset_index(name='Count of New Songs')

# Sort by Year and Month in ascending order
new_songs_by_month_sorted = new_songs_by_month.sort_values(by=['Year', 'Month'], ascending=True)

# Display the final DataFrame
print(new_songs_by_month_sorted)

    Year Month  Count of New Songs
0   2022    01                 103
1   2022    02                  54
2   2022    03                  39
3   2022    04                  57
4   2022    05                  89
5   2022    06                  61
6   2022    07                  68
7   2022    08                  44
8   2022    09                  58
9   2022    10                  58
10  2022    11                  60
11  2022    12                  85
12  2023    01                  66
13  2023    02                  44
14  2023    03                  70
15  2023    04                  56
16  2023    05                  33
17  2023    06                  56
18  2023    07                 107
19  2023    08                  67
20  2023    09                  76
21  2023    10                  80
22  2023    11                  69
23  2023    12                  77
24  2024    01                  75
25  2024    02                  53
26  2024    03                  58
27  2024    04      

In [26]:
# Monthly New Songs in Hot 100 (2022-2024)
# Create the heatmap
fig = px.density_heatmap(new_songs_by_month_sorted,
                          x='Month',
                          y='Year',
                          z='Count of New Songs',
                          title='Monthly New Songs in Hot 100 (2022-2024)',
                          labels={'Count of New Songs': 'Number of New Songs'})

fig.update_layout(
    title=dict(
        text='Monthly New Songs in Hot 100 (2022-2024)',
        font=dict(size=24),  
        x=0.5
    )
)

# Show the heatmap
fig.show()

Insights

The data on the count of new songs entering the Billboard Hot 100 from 2022 to 2024 reveals interesting trends in music release patterns. January 2022 saw a significant surge with 103 new songs, possibly reflecting the post-holiday release strategy where artists capitalize on fresh starts and listener engagement after the holiday season.

Throughout 2022, the monthly counts varied, with notable peaks in May (89 songs) and December (85 songs). These fluctuations may indicate strategic releases aligned with summer and holiday listening trends, as artists often aim to capture heightened listener interest during these periods.

In 2023, there was a notable spike in July, with 107 new songs, suggesting a possible summer release wave where artists aim to capitalize on seasonal listening habits. The months that followed maintained a steady influx of new music, with counts remaining consistently in the 60s to 80s.

The trend continued into 2024, where the monthly counts generally ranged between 36 and 76 new songs, indicating a slight decline in new releases as the year progressed. This decrease may reflect a natural ebb in the release cycle or shifts in artist strategies.

Overall, this data highlights the cyclical nature of music releases, with certain months showing stronger activity likely tied to listener behavior and industry strategies. The trends suggest that both seasonal factors and strategic timing play crucial roles in the volume of new music entering the charts.